## Setup import

In [20]:
import pandas as pd 
import wikipedia as wp

import requests
import io

## Use the Notebook to build the code to scrape the following Wikipedia page

In [21]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

## Determine the index of your table

In [6]:
df = pd.read_html(html, header = 0)[0]

## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [23]:
df = df[df.Borough != 'Not assigned']

In [24]:
df = df.groupby(['Postal code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [25]:
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

## In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [26]:
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## Rename the first column to allow merging dataframes on Postcode

In [27]:
c.columns = ['Postal code', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postal code')

## Reorder column names and show the dataframe

In [28]:
df = df[['Postal code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df

Postal code      Borough  \
0           M1B  Scarborough   
1           M1C  Scarborough   
2           M1E  Scarborough   
3           M1G  Scarborough   
4           M1H  Scarborough   
..          ...          ...   
98          M9N         York   
99          M9P    Etobicoke   
100         M9R    Etobicoke   
101         M9V    Etobicoke   
102         M9W    Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                      Malvern / Rouge  43.806686 -79.194353  
1             Rouge Hill / Port Union / Highland Creek  43.784535 -79.160497  
2                  Guildwood / Morningside / West Hill  43.763573 -79.188711  
3                                               Woburn  43.770992 -79.216917  
4                                            Cedarbrae  43.773136 -79.239476  
..                                                 ...        ...        ...  
98                                              Weston  43.706876 -79.518188  
99                                           Westmount  43.696319 -79.532242  
100  Kingsview Village / St. Phillips / Martin Grov...  43.688905 -79.554724  
101  South Steeles / Silverstone / Humbergate / Jam...  43.739416 -79.588437  
102                                          Northwest  43.706748 -79.594054  

[103 rows x 5 columns]